# Generate adults dataset from MIMIC III using BigQuery

## Libraries and Environment

In [1]:
# Import libraries
from datetime import datetime
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import drive
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()
# mount
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Set up environment variables
project_id = 'mimic-370210'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# test it works
df = run_query("""
SELECT *
FROM `mimic-370210.adults.adults_id`
""")
df

,subject_id
0,9
1,21
2,38
3,61
4,88
...,...
20931,99955
20932,99957
20933,99985
20934,99991


## Query static information

In [8]:
adults_id = run_query("""
SELECT *
FROM `mimic-370210.adults.adults_id`
""")
adults_id.head()

,subject_id
0,9
1,21
2,38
3,61
4,88


In [9]:
adults_static = run_query("""
SELECT *
FROM `mimic-370210.adults.adults_static`
""")
adults_static.head()

,subject_id,gender,age,spec,diagnoses,time
0,3,M,76.0,BLOOD CULTURE,False,2101-10-20 18:00:00
1,3,M,76.0,BLOOD CULTURE,False,2101-10-20 18:30:00
2,3,M,76.0,BLOOD CULTURE,False,2101-10-21 04:00:00
3,3,M,76.0,BLOOD CULTURE,False,2101-10-21 04:00:00
4,3,M,76.0,BLOOD CULTURE,False,2101-10-21 04:15:00


In [ ]:
# adults_event = run_query("""
# SELECT *
# FROM `mimic-370210.adults.adults_event`
# """)

**Don't read neonates_event NOW**\
**Read it use loop**

In [10]:
# Function: to perforrm OR operation considering NaN
def f_or(a,b):
    if not np.isnan(a):
        return a
    if not np.isnan(b):
        return b
    return np.nan

In [13]:

# Function: re-arranege the dataset according to the timepoints
# Input: the DataFrames of static_info and event_info
# Outout: the re-arranged DataFrame


def arrange_time(df_static, df_events, all_columns):
    df_final = pd.DataFrame(columns=all_columns)
    sepsis_indicator = False  #used for storing neonates in different folders
    # read the static data
    subject_id = df_static.subject_id[0]
    gender = df_static.gender[0]
    age = df_static.age[0]
    all_events = df_events
    all_events['sepsis'] = False  #add a new column of sepsis
    for index, row in df_static.iterrows():
      if row['diagnoses'] == "True":
        sepsis_indicator = True
        sepsis_onset_time = row['time']  #only pick the first on-set-time
        all_events = all_events.append({'subject_id':subject_id, 'time':sepsis_onset_time,
                            'value':np.nan,'unit':np.nan, 'abbr':np.nan, 'sepsis':True},
                          ignore_index = True)
        all_events.sort_values('time', inplace = True)
        break #keep the first time of sepsis if there are several

    # Get the time points from events
    time_sequence = all_events.loc[:,"time"]
    time_sequence.drop_duplicates( keep='first',inplace= True)
    time_sequence = time_sequence.reset_index(drop =True)
    len_time = len(time_sequence)
    for index_time, value in time_sequence.items():
        time = str(value)
        df_current = all_events
        df_current = df_current[df_current['time'].astype(str).str.fullmatch(time)]
        # print('Aviliable DataFrame at this time point',df_current)
        # iterrate each row and set values
        sepsis_final = False
        NBPs_final = np.nan
        NBPd_final = np.nan
        NBPm_final = np.nan
        temp_final = np.nan
        hr_final = np.nan
        rr_final = np.nan
        abe_final = np.nan
        FiO2_final = np.nan
        SpO2_final = np.nan
        PCO2_final = np.nan
        ph_final = np.nan
        magnesium_final = np.nan
        calcium_final = np.nan
        latic_final  = np.nan
        sodium_final = np.nan
        potassium_final = np.nan
        creatinine_final = np.nan
        chloride_final = np.nan
        HCO3_final = np.nan
        glucose_final = np.nan
        WBC_final = np.nan
        BUN_final = np.nan
        PTT_final = np.nan
        platelet_final = np.nan
        for index, row in df_current.iterrows():
            if row['sepsis']:
              sepsis_final = True # sepsis positive at current time

            if row['abbr'] == 'SBP':
                NBPs = row['value']
            else: NBPs = np.nan
            NBPs_final = f_or(NBPs_final,NBPs)
            
            if row['abbr'] == 'DBP':
                NBPd = row['value']
            else: NBPd = np.nan
            NBPd_final = f_or(NBPd_final,NBPd)
            
            if row['abbr'] == 'MAP':
                NBPm = row['value']
            else: NBPm = np.nan
            NBPm_final = f_or(NBPm_final,NBPm)
            
            if row['abbr'] == 'Temp':
                temp = row['value']
            else: temp = np.nan
            temp_final = f_or(temp_final,temp)
            
            if row['abbr'] == 'HR':
                hr = row['value']
            else: hr = np.nan
            hr_final = f_or(hr_final,hr)
            
            if row['abbr'] == 'RR':
                rr = row['value']
            else: rr = np.nan
            rr_final = f_or(rr_final,rr)
            
            if row['abbr'] == 'BaseExcess':
                abe = row['value']
            else: abe = np.nan
            abe_final = f_or(abe_final,abe)
            
            # if row['abbr'] == 'FiO2':
            #     FiO2 = row['value']
            # else: FiO2 = np.nan
            # FiO2_final = f_or(FiO2_final,FiO2)

            if row['abbr'] == 'SaO2':
                SpO2 = row['value']
            else: SpO2 = np.nan
            SpO2_final = f_or(SpO2_final,SpO2)
            
            # if row['abbr'] == 'PCO2':
            #     PCO2 = row['value']
            # else: PCO2 = np.nan
            # PCO2_final = f_or(PCO2_final,PCO2)
            
            if row['abbr'] == 'PH':
                ph = row['value']
            else: ph = np.nan
            ph_final = f_or(ph_final,ph)
            
            if row['abbr'] == 'Magnesium':
                magnesium = row['value']
            else: magnesium = np.nan
            magnesium_final = f_or(magnesium_final,magnesium)
            
            if row['abbr'] == 'Calcium':
                calcium = row['value']
            else: calcium = np.nan
            calcium_final = f_or(calcium_final,calcium)
            
            if row['abbr'] == 'Lactic':
                latic = row['value']
            else: latic = np.nan
            latic_final = f_or(latic_final,latic)
            
            if row['abbr'] == 'Sodium':
                sodium = row['value']
            else: sodium = np.nan
            sodium_final = f_or(sodium_final,sodium)
            
            if row['abbr'] == 'Potassium':
                potassium = row['value']
            else: potassium = np.nan
            potassium_final = f_or(potassium_final,potassium)
            
            if row['abbr'] == 'Creatinine':
                creatinine = row['value']
            else: creatinine = np.nan
            creatinine_final = f_or(creatinine_final,creatinine)
            
            if row['abbr'] == 'Chloride':
                chloride = row['value']
            else: chloride = np.nan
            chloride_final = f_or(chloride_final,chloride)
            
            if row['abbr'] == 'HCO3':
                HCO3 = row['value']
            else: HCO3 = np.nan
            HCO3_final = f_or(HCO3_final,HCO3)
            
            if row['abbr'] == 'Glucose':
                glucose = row['value']
            else: glucose = np.nan
            glucose_final = f_or(glucose_final,glucose)
            
            if row['abbr'] == 'WBC':
                WBC = row['value']
            else: WBC = np.nan
            WBC_final = f_or(WBC_final,WBC)
            
            if row['abbr'] == 'BUN':
                BUN = row['value']
            else: BUN = np.nan
            BUN_final = f_or(BUN_final,BUN)
            
            if row['abbr'] == 'PTT':
                PTT = row['value']
            else: PTT = np.nan
            PTT_final = f_or(PTT_final,PTT)
            
            if row['abbr'] == 'Platelet':
                platelet = row['value']
            else: platelet = np.nan
            platelet_final = f_or(platelet_final,platelet)

        new_row={'sepsis':sepsis_final,'subject_id':subject_id, 'gender':gender, 'age':age,
                'SBP':NBPs_final,'DBP':NBPd_final,'MAP':NBPm_final,
                 'Temp':temp_final,'HR':hr_final,'RR':rr_final,
                 'BaseExcess':abe_final,'SaO2':SpO2_final,
                 'PH':ph_final, 'Magnesium':magnesium_final,'Calcium':calcium_final,
                 'Lactic':latic_final,'Sodium':sodium_final,'Potassium':potassium_final,
                'Creatinine':creatinine_final,'Chloride':chloride_final,'HCO3':HCO3_final,
                 'Glucose':glucose_final, 'WBC':WBC_final, 'BUN':BUN_final,
                 'PTT': PTT_final,'Platelet':platelet_final,'time':time}
        # print('The new row:',new_row)
        # new_row=pd.Series(new_row)
        df_final = df_final.append(new_row, ignore_index=True)
 
    return df_final, sepsis_indicator
    

## Main function

In [14]:
# Main

# compared with the adults, age is removed and los is unavailable.
all_columns = ['sepsis','subject_id', 'gender','age',
                'SBP','DBP','MAP','Temp','HR','RR','BaseExcess',
                'SaO2','PH','Magnesium','Lactic',
                'Calcium','Sodium','Potassium','HCO3',
                'Creatinine','Chloride', 'Glucose',
                'WBC','BUN','PTT','Platelet','time']   # the features from events are renamed


for index, row in adults_id.iterrows():
    index += 1
    # if index <=6343: continue
    print('-----------Processing the {}th/20936 adult-----------'.format(index))

    current_id = row['subject_id']

    # to generate the static dataset 
    query_static = f"""
      SELECT *
      FROM `mimic-370210.adults.adults_static`
      WHERE subject_id = {current_id}
    """
    static_info = run_query(query_static)
    static_info.drop_duplicates(keep = 'first', inplace = True)

    # to generate the event dataset
    query_event = f"""
    SELECT *
    FROM `mimic-370210.adults.adults_event`
    WHERE subject_id = {current_id}
    """
    event_info = run_query(query_event)
    event_info.drop_duplicates(keep = 'first', inplace = True)

    # to generate the datasheet in one table
    all_info, sepsis_indicator = arrange_time(static_info,event_info,all_columns)

    if len(all_info) < 10:
      print('Fewer than 10 points -> Discard')
      continue

    print(all_info.subject_id[0], sepsis_indicator)
    if sepsis_indicator:
        all_info.to_csv('/content/drive/MyDrive/MIMIC/Results/adults/sepsis/'+str(all_info.subject_id[0])+'.csv', index = False)
    else:
        all_info.to_csv('/content/drive/MyDrive/MIMIC/Results/adults/non_sepsis/'+str(all_info.subject_id[0])+'.csv', index = False)

-----------Processing the 1th/20936 adult-----------
9 True
-----------Processing the 2th/20936 adult-----------
21 True
-----------Processing the 3th/20936 adult-----------
38 True
-----------Processing the 4th/20936 adult-----------
61 True
-----------Processing the 5th/20936 adult-----------
88 True
-----------Processing the 6th/20936 adult-----------
109 True
-----------Processing the 7th/20936 adult-----------
111 True
-----------Processing the 8th/20936 adult-----------
124 True
-----------Processing the 9th/20936 adult-----------
130 True
-----------Processing the 10th/20936 adult-----------
145 True
-----------Processing the 11th/20936 adult-----------
161 True
-----------Processing the 12th/20936 adult-----------
166 True
-----------Processing the 13th/20936 adult-----------
177 True
-----------Processing the 14th/20936 adult-----------
187 True
-----------Processing the 15th/20936 adult-----------
188 True
-----------Processing the 16th/20936 adult-----------
191 True
-------

GenericGBQException: ignored